In [1]:
from sympy import symbols, diff, solve, Transpose, Integer, Poly
from fractions import Fraction
import math
import numpy as np
import pandas as pd
import decimal
import fractions
from sympy.matrices import *
import random as random
import math as m
from IPython.display import display, HTML

In [2]:
def matrizJacobiana(f, variables):
    jacobiana = []
    for i in range(0,len(variables)):
        jacobiana.append(diff(f, variables[i]))
    return jacobiana

def matrizJacobianaDic(f, variables):
    jacobiana = {}
    i = 0
    for v in variables:
        jacobiana[v] = diff(f, variables[i])
        i = i + 1
    return jacobiana

def matrizHessiana(f, variables):
    hessiana = []
    for fi in f:
        hessiana.append(matrizJacobiana(fi, variables))
    return hessiana
    
def createVariableDictionary(variables):
    result = {}
    for v in variables:
        result[v] = 0
    return result

def matToDic(mat, variables):
    ml = mat.tolist()
    dic = {}
    i = 0
    for var in variables:
        dic[var] = ml[i][0]
        i = i +1
    return dic
    
def generateRandomPointInLimits(variables, limits):
    result = createVariableDictionary(variables)
    for j in range(0, len(variables)):
        result[limits[j][0]] = limits[j][1] + (limits[j][2] - limits[j][1])*random.uniform(0, 1)
    return result.copy()

def createMats(functions, variables, initPoint):
    A = list()
    b = list()
    x = list(initPoint.values())
    for function in functions:
        coefs = list()
        p = Poly(function, function.free_symbols)
        c = p.coeffs()
        fs = function.free_symbols
        if(len(c) == len(fs)):
            b.append(0)
        else:
            b.append(c[len(c)-1])
        for v in variables:
            coefs.append(function.coeff(v))
        A.append(coefs)
    return (Matrix(A), Transpose(Matrix([b])), Transpose(Matrix([x])))

def roundValues(dic):
    for i in dic:
        dic[i] = round(dic[i],4)
    return dic

In [3]:
def randomSearch(f, variables, limits, iteraciones, metodo, epsilon, initPoint):
    maxF = f.evalf(subs=initPoint)
    error = "init"
    i = 0
    result = createVariableDictionary(variables)
    maxResults = result.copy()
    values = pd.DataFrame(columns=('x_v', 'f(x_v)'))
    values.loc[len(values)] = [roundValues(initPoint.copy()), round(maxF,4)]
    while ((i < iteraciones) and (error == "init" or error > epsilon)):
        result = generateRandomPointInLimits(variables, limits)
        fn = f.evalf(subs=result)
        if (maxF == "init") or (((fn > maxF) and (metodo == "maximo")) or ((fn < maxF) and (metodo == "minimo"))):
            if (maxF is not "init"):
                error = m.fabs(fn - maxF)
            maxF = fn
            maxResults = result.copy()
        values.loc[len(values)] = [roundValues(maxResults.copy()), round(maxF,4)]
        i = i + 1
    return (maxResults,values)

In [4]:
h = symbols('h')
    
def funcGradiente(a, dx, metodo):
    if (metodo == "maximo"):
        return a + dx*h
    elif (metodo == "minimo"):
        return a - dx*h
    
def gradienteFijo(f, variables, limits, metodo):
    mJ = matrizJacobiana(f, variables)
    mH = matrizHessiana(mJ, variables)
    solucion = solve(mJ, dict=True)
    determinante = Matrix(mH).det()
    values = pd.DataFrame(columns=('x_v', 'f(x_v)'))
    if (determinante >= 0) and ((mH[0][0] >= 0 and metodo is "minimo") or (mH[0][0] <= 0 and metodo is "maximo")):
        isInIntervale = True
        for myLimits in limits:
            mySolution = solucion[0].get(myLimits[0])
            if not(myLimits[1]<=mySolution<=myLimits[2]):
                isInIntervale = False
                break
        if isInIntervale:
            values.loc[len(values)] = [solucion[0], f.evalf(subs=solucion[0])]
            return solucion[0]
    return solucion[0]

def gradienteAdaptativoMetodo1(f, variables, limits, iteraciones, metodo, epsilon, initPoint):
    result = initPoint
    mJ = matrizJacobianaDic(f, variables)
    f_x0 = f.evalf(subs=result)
    error = "init"
    f_d = {}
    i = 0
    values = pd.DataFrame(columns=('x_v', 'f(x_v)'))
    values.loc[len(values)] = [roundValues(result.copy()), round(f_x0, 4)]
    while (i < iteraciones) and ( (error is "init") or (error > epsilon) ):
        for v in variables:
            f_d[v] = funcGradiente(result[v], mJ[v].evalf(subs=result), metodo)
        g = f.subs(f_d)
        initRandomPoint = generateRandomPointInLimits((h,), [[h,-10,10]])
        val_h , resp = randomSearch(g, (h, ), [[h,-10,10]], 1000, metodo, 0.01, initRandomPoint)
        for v in variables:
            result[v] = f_d[v].evalf(subs=val_h)
        f_x1 = f.evalf(subs=result)
        error = m.fabs(f_x1 - f_x0)
        f_x0 = f_x1
        values.loc[len(values)] = [roundValues(result.copy()), round(f_x0, 4)]
        i = i +1
    return (result, values)
    
def gradienteDescendente(f, variables, limits, iteraciones, metodo, epsilon, tasa, initPoint):
    result = initPoint
    mJ = matrizJacobianaDic(f, variables)
    f_x0 = f.evalf(subs=result)
    error = "init"
    i = 0
    values = pd.DataFrame(columns=('x_v', 'f(x_v)'))
    values.loc[len(values)] = [roundValues(result.copy()), round(f_x0, 4)]
    while (i < iteraciones) and ( (error is "init") or (error > epsilon) ):
        for v in variables:
            result[v] = funcGradiente(result[v], mJ[v].evalf(subs=result), metodo).evalf(subs={h: tasa})
        f_x1 = f.evalf(subs=result)
        error = m.fabs(f_x1 - f_x0)
        f_x0 = f_x1
        values.loc[len(values)] = [roundValues(result.copy()), round(f_x0, 4)]
        i = i+1
    return (result, values)

def gradienteConjugado(f, variables, limits, iteraciones, metodo, epsilon, initPoint):
    mJ = matrizJacobiana(f, variables)
    A, b, x = createMats(mJ, variables, initPoint)
    r_0 = b - A*x
    p = r_0
    k = 0
    values = pd.DataFrame(columns=('x_v', 'f(x_v)'))
    values.loc[len(values)] = [roundValues(initPoint), round(f.evalf(subs=initPoint), 4)]
    for i in range(0, iteraciones):
        alfa = (Transpose(r_0)*r_0)[0]/(Transpose(p)*A*p)[0]
        x = x + alfa*p
        r_1 = r_0 - alfa*A*p
        error = (Transpose(r_1) * r_1)[0]
        varInDic = matToDic(-x, variables)
        values.loc[len(values)] = [roundValues(varInDic), round(f.evalf(subs=varInDic), 4)]
        if error < epsilon:
            return (x, values)
        b = (Transpose(r_1)*r_1)[0]/(Transpose(r_0)*r_0)[0]
        p = r_1 + b*p
        r_0 = r_1
    return (-x, values)

In [5]:
def optimization(f, variables, limits, iteraciones, metodo, epsilon, optimFunc, tasa, initPoint):
    if(optimFunc == "gradienteDescendente"):
        return gradienteDescendente(f, variables, limits, iteraciones, metodo, epsilon, tasa, initPoint)
    if(optimFunc == "gradienteAdaptativoMetodo1"):
        return gradienteAdaptativoMetodo1(f, variables, limits, iteraciones, metodo, epsilon, initPoint)
    if(optimFunc == "gradienteConjugado"):
        return gradienteConjugado(f, variables, limits, iteraciones, metodo, epsilon, initPoint)
    if(optimFunc == "randomSearch"):
        return randomSearch(f, variables, limits, iteraciones, metodo, epsilon)

In [6]:
def pruebaRandom(f, variables, limits, iteraciones, metodo, epsilon, initPoint):
    folds = 5
    df = []
    for i in range(0, folds):
        resp1, table = randomSearch(f, variables, limits, iteraciones, metodo, epsilon, initPoint)
        df.append(table)
    return pd.concat(df, axis=1)

In [7]:
def evalOptimization(f, variables, limits, iteraciones, metodo, epsilon, tasa):
    initPoint = generateRandomPointInLimits(variables, limits)
    #for function in functions:
    #evaluacion de metodos basados en gradiente
    resp1, table1 = optimization(f, variables, limits, iteraciones, metodo, epsilon, "gradienteDescendente", tasa, initPoint.copy())
    resp2, table2 = optimization(f, variables, limits, iteraciones, metodo, epsilon, "gradienteAdaptativoMetodo1", tasa, initPoint.copy())
    resp3, table3 = optimization(f, variables, limits, iteraciones, metodo, epsilon, "gradienteConjugado", tasa, initPoint.copy())
    #evaluacion de metodos no basados en gradiente
    table4 = pruebaRandom(f, variables, limits, iteraciones, metodo, epsilon, initPoint.copy())
    pd.set_option('display.max_colwidth', -1)
    return pd.concat([table1, table2, table3, table4], axis=1, keys=['Gradiente fijo', 'Gradiente adaptativo', 'Gradiente conjugado', 'Busqueda aleatoria'])

In [32]:
def main():
    x, y, z, w, v= symbols('x y z w v', real=True)
    f0 = -x**2+x-2*y**2-3*z**2 + 2*x*y- 3*x - 45*z
    f1 = x**2+x+2*y**2+3*z**2 #minimizar
    f2 = x**2+y**2+5*z**2+x*2-5*y+6*z+6 #minimizar
    f3 = x**2+y**2+5*z**2+x*2-5*y+6*z+6*w**2-9*w+v**2-3*v #minimizar
    f4 = -x**2-y**2-z**2-w**2-v**2 + 3*x + 985*y +45- 45*w +7*v #maximizar
    iteraciones = 100
    
    resp = []
    resp.append(evalOptimization(f0, (x,y,z), [[x, -2, 10],[y, -2, 10], [z, -10 , 10]], iteraciones, "maximo", 0.01, 0.02))
    resp.append(evalOptimization(f1, (x,y,z), [[x, -2, 10],[y, -2, 10], [z, -10 , 10]], iteraciones, "minimo", 0.01, 0.02))
    resp.append(evalOptimization(f2, (x,y,z), [[x, -2, 10],[y, -2, 10], [z, -10 , 10]], iteraciones, "minimo", 0.01, 0.02))
    resp.append(evalOptimization(f3, (x,y,z,w,v), [[x, -2, 10],[y, -2, 10], [z, -10 , 10], [w,-10,10], [v,-10,10]], iteraciones, "minimo", 0.001, 0.02))
    resp.append(evalOptimization(f4, (x,y,z,w,v), [[x, -2, 10],[y, -2, 10], [z, -10 , 10], [w,-10,10], [v,-10,10]], iteraciones, "maximo", 0.001, 0.02))

    return HTML("""<h5> Función: """ + str(f0) + """</h5>
        <h2>Historial</h2>
        """ + resp[0].to_html() + 
        """"<h5> Función: """ + str(f1) + """</h5>
        <h2>Historial</h2>
        """ + resp[1].to_html() +
        """"<h5> Función: """ + str(f2) + """</h5>
        <h2>Historial</h2>
        """ + resp[2].to_html() +
        """"<h5> Función: """ + str(f3) + """</h5>
        <h2>Historial</h2>
        """ + resp[3].to_html() +
        """"<h5> Función: """ + str(f4) + """</h5>
        <h2>Historial</h2>
        """ + resp[4].to_html())

In [33]:
main()

In [41]:
x, y, z, w, v= symbols('x y z w v', real=True)
f0 = -x**2+x-2*y**2-3*z**2 + 2*x*y- 3*x - 45*z
f1 = x**2+x+2*y**2+3*z**2 #minimizar
f2 = x**2+y**2+5*z**2+x*2-5*y+6*z+6 #minimizar
f3 = x**2+y**2+5*z**2+x*2-5*y+6*z+6*w**2-9*w+v**2-3*v #minimizar
f4 = -x**2-y**2-z**2-w**2-v**2 + 3*x + 985*y +45- 45*w +7*v #maximizar


table2= evalOptimization(f1, (x,y,z), [[x, -2, 10],[y, -2, 10], [z, -10 , 10]], 200, "minimo", 0.001, 0.01)
table2

Gradiente fijo           \
                                     x_v   f(x_v)   
0    {x: 2.6007, y: -1.0045, z: -2.769}   34.3845   
1    {x: 2.5387, y: -0.9643, z: -2.6029}  31.1681   
2    {x: 2.4779, y: -0.9257, z: -2.4467}  28.2909   
3    {x: 2.4184, y: -0.8887, z: -2.2999}  25.7149   
4    {x: 2.36, y: -0.8531, z: -2.1619}    23.4067   
5    {x: 2.3028, y: -0.819, z: -2.0322}   21.3366   
6    {x: 2.2467, y: -0.7862, z: -1.9103}  19.4782   
7    {x: 2.1918, y: -0.7548, z: -1.7956}  17.8082   
8    {x: 2.138, y: -0.7246, z: -1.6879}   16.3060   
9    {x: 2.0852, y: -0.6956, z: -1.5866}  14.9533   
10   {x: 2.0335, y: -0.6678, z: -1.4914}  13.7337   
11   {x: 1.9828, y: -0.6411, z: -1.4019}  12.6328   
12   {x: 1.9332, y: -0.6154, z: -1.3178}  11.6379   
13   {x: 1.8845, y: -0.5908, z: -1.2388}  10.7376   
14   {x: 1.8368, y: -0.5672, z: -1.1644}  9.9219    
15   {x: 1.7901, y: -0.5445, z: -1.0946}  9.1817    
16   {x: 1.7443, y: -0.5227, z: -1.0289}  8.5092    
17   {x: 1.6994, y: -0.5018, z: -0.9672}  7.8972    
18   {x: 1.6554, y: -0.4817, z: -0.9091}  7.3395    
19   {x: 1.6123, y: -0.4625, z: -0.8546}  6.8305    
20   {x: 1.5701, y: -0.444, z: -0.8033}   6.3653    
21   {x: 1.5287, y: -0.4262, z: -0.7551}  5.9393    
22   {x: 1.4881, y: -0.4092, z: -0.7098}  5.5488    
23   {x: 1.4483, y: -0.3928, z: -0.6672}  5.1901    
24   {x: 1.4094, y: -0.3771, z: -0.6272}  4.8601    
25   {x: 1.3712, y: -0.362, z: -0.5895}   4.5561    
26   {x: 1.3337, y: -0.3475, z: -0.5542}  4.2755    
27   {x: 1.2971, y: -0.3336, z: -0.5209}  4.0162    
28   {x: 1.2611, y: -0.3203, z: -0.4897}  3.7761    
29   {x: 1.2259, y: -0.3075, z: -0.4603}  3.5534    
..                                   ...     ...    
171  NaN                                 NaN        
172  NaN                                 NaN        
173  NaN                                 NaN        
174  NaN                                 NaN        
175  NaN                                 NaN        
176  NaN                                 NaN        
177  NaN                                 NaN        
178  NaN                                 NaN        
179  NaN                                 NaN        
180  NaN                                 NaN        
181  NaN                                 NaN        
182  NaN                                 NaN        
183  NaN                                 NaN        
184  NaN                                 NaN        
185  NaN                                 NaN        
186  NaN                                 NaN        
187  NaN                                 NaN        
188  NaN                                 NaN        
189  NaN                                 NaN        
190  NaN                                 NaN        
191  NaN                                 NaN        
192  NaN                                 NaN        
193  NaN                                 NaN        
194  NaN                                 NaN        
195  NaN                                 NaN        
196  NaN                                 NaN        
197  NaN                                 NaN        
198  NaN                                 NaN        
199  NaN                                 NaN        
200  NaN                                 NaN        

                    Gradiente adaptativo           \
                                     x_v   f(x_v)   
0    {x: 2.6007, y: -1.0045, z: -2.769}   34.3845   
1    {x: 1.4756, y: -0.2755, z: 0.2452}   3.9853    
2    {x: 0.0172, y: 0.1313, z: -0.2978}   0.3181    
3    {x: -0.1862, y: 0.028, z: 0.0535}   -0.1414    
4    {x: -0.436, y: -0.0166, z: -0.0743} -0.2288    
5    {x: -0.4601, y: -0.0041, z: 0.0096} -0.2481    
6    {x: -0.5957, y: 0.0238, z: -0.0885} -0.2162    
7    {x: -0.5699, y: 0.0109, z: -0.0168} -0.2440    
8    {x: -0.5216, y: -0.0042, z: 0.0181} -0.2485    
9    {x: -0.4883, y: 0.0087, z: -0.0655} -0.2369    
10   {x: -0.4909, y: 0.0049, z: -0.0222} -0.2484  

In [143]:
x, y, z, w, v= symbols('x y z w v', real=True)
f1 = x**2+x+2*y**2+3*z**2 #minimizar

pruebaRandom(f1, (x,y,z), [[x, -2, 2],[y, -2, 2], [z, -2 , 2]], 1000, "minimo", 0.001)

Prueba1           \
                                       x_v   f(x_v)   
0     {x: -1.7013, y: -0.826, z: -1.9461}   13.9197   
1     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
2     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
3     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
4     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
5     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
6     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
7     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
8     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
9     {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
10    {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
11    {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
12    {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
13    {x: 1.3873, y: -0.9579, z: -0.138}    5.2042    
14    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
15    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
16    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
17    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
18    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
19    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
20    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
21    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
22    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
23    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
24    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
25    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
26    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
27    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
28    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
29    {x: -0.6361, y: -0.5626, z: -0.6312}  1.5968    
...                                    ...     ...    
971   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
972   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
973   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
974   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
975   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
976   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
977   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
978   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
979   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
980   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
981   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
982   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
983   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
984   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
985   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
986   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
987   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
988   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
989   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
990   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
991   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
992   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
993   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
994   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
995   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
996   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
997   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
998   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
999   {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    
1000  {x: -0.2284, y: -0.1693, z: -0.0239} -0.1172    

                                  Prueba2           \
                                      x_v   f(x_v)   
0     {x: -1.7013, y: -0.826, z: -1.9461}  13.9197   
1     {x: 1.5011, y: 1.1558, z: 0.5165}    7.2264    
2     {x: 1.5011, y: 1.1558, z: 0.5165}    7.2264    
3     {x: -1.7898, y: -0.9184, z: 0.9707}  5.9273    
4     {x: 0.4247, y: -0.978, z: -0.9341}   5.1352    
5     {x: 0.4247, y: -0.978, z: -0.9341}   5.1352    
6     {x: 0.4247, y: -0.978, z: -0.9341}   5.1352    
7     {x: 0.4247, y: -0.978, z: -0.9341}   5.1352    
8     {x: 0.4247, y: -0.978, 